In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from urllib.parse import urlparse, urljoin

    
def get_clear_text(url):
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    
    # Удаление элементов "скрипт" и "стиль". У всех сайтов, которые проверял, не было элементов "стиль", но пускай будет.
    # UPD: а нет, всё же есть.
    for script_style in soup(["script", "style"]):
        script_style.extract()
        
    # Проверка показала, что get_text() не считывает содержимое эелементов "script" и "style",
    # так что большая ли необходимость в удалении этих эелементов???     
    text = soup.get_text()
    
    # Разбить на строки без пустых пространств.
    lines = (line.strip() for line in text.splitlines())
    
    # Разделение линий с 2-мя и более пробелами.
    clear_lines = (phrase.strip() for line in lines for phrase in line.split(r'\s{2,}'))
    
    # Объединяем все линии так, что пустые строки исчезают.
    text = '\n'.join(clear_line for clear_line in clear_lines if clear_line)
    return(text)


internal_urls = set()
external_urls = set()
urls=set()


def is_valid(url):
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)


def get_all_website_links(url):
    urls = set()
    
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            continue
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_valid(href):
            continue
        if href in internal_urls:
            continue
        if domain_name not in href:
            if href not in external_urls:
                external_urls.add(href)
            continue
        urls.add(href)
        internal_urls.add(href)
    return urls
    
total_urls_visited = 0
url_counter=0


def crawler(url, max_urls=100):
    global total_urls_visited
    total_urls_visited += 1
    links = get_all_website_links(url)
    if len(links) < max_urls:
        for link in links:
            total_urls_visited += len(links) 
            if total_urls_visited > max_urls:
                break
            crawler(link, max_urls=max_urls)
    for link in links:
        global url_counter
        if url_counter==150:
            break
        url_counter+=1
        urls.add(link)
        
page_counter=0
     
    
def write_txt(text, url, index_txt):
    global page_counter
    f = open(f'index{page_counter}.txt','w', encoding="utf-8") 
    
    # Выкачиваем текст в отдельный файл.
    f.write(text) 
    f.close()
    # Заполняем index.txt новой строкой.
    index_txt.write(f'{page_counter}:%s\n' % url)      
    page_counter+=1
        
        
if __name__ == "__main__":
    url = "https://en.wikipedia.org/wiki/Main_Page"
    n=get_clear_text(url)
    index_txt = open(f'index.txt','w', encoding="utf-8")
    write_txt(n,url, index_txt)
    crawler(url)

    for u in urls:
        text=get_clear_text(u)
        
        # На случай, если на странице нет текста.
        if len(text)==0:
            continue
        write_txt(text,u, index_txt)
    index_txt.close()
        
